In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.models import Sequential, load_model
from keras.utils import load_img, img_to_array
from sklearn.model_selection import StratifiedKFold

IMAGE_DIMENSION = 256
BATCH_SIZE = 32
FILTERS = 32
POOL_SIZE = (2, 2)
FIELD_SIZE = 3
ACTIVATION_FUNCTION = "relu"
DROPOUT = 0.3
UNITS = 256

def create_model():
    model = Sequential()
    model.add(Conv2D(FILTERS, FIELD_SIZE, input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3), activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(MaxPooling2D(pool_size=POOL_SIZE))
    model.add(BatchNormalization())
    model.add(Conv2D(FILTERS, FIELD_SIZE, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(MaxPooling2D(pool_size=POOL_SIZE))
    model.add(BatchNormalization())
    model.add(Conv2D(FILTERS, FIELD_SIZE, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(MaxPooling2D(pool_size=POOL_SIZE))
    model.add(BatchNormalization())
    model.add(Conv2D(FILTERS, FIELD_SIZE, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(MaxPooling2D(pool_size=POOL_SIZE))
    model.add(BatchNormalization())
    model.add(Conv2D(FILTERS, FIELD_SIZE, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(MaxPooling2D(pool_size=POOL_SIZE))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS // 4, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS // 2, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS // 4, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS // 2, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS // 4, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS // 2, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS // 4, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS // 2, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS // 4, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS // 2, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS // 4, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS // 2, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(DROPOUT))
    model.add(BatchNormalization())
    model.add(Dense(units=UNITS, activation=ACTIVATION_FUNCTION))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(units=1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

IMAGE_SIZE: tuple[int, int] = (IMAGE_DIMENSION, IMAGE_DIMENSION)
TRAIN_DIR: str = "imagensCancerMama/treino_dataset"

dataframe_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
dataframe_data = dataframe_generator.flow_from_directory(TRAIN_DIR,
                                                  shuffle=False,
                                                  target_size=IMAGE_SIZE,
                                                  batch_size=BATCH_SIZE,
                                                  class_mode="binary"
                                                )
dataframe = pd.DataFrame({
    "filepaths": dataframe_data.filepaths,
    "labels": dataframe_data.labels.astype(str)
})

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

models_accuracies: list[tuple[float, float]] = []
sk_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
fold = 1
for train, val in sk_fold.split(dataframe["filepaths"], dataframe["labels"]):
    train_dataframe = dataframe.iloc[train].reset_index(drop=True)
    val_dataframe = dataframe.iloc[val].reset_index(drop=True)

    train_dataset = train_datagen.flow_from_dataframe(
        dataframe=train_dataframe,
        directory=None,
        x_col="filepaths",
        y_col="labels",
        target_size=IMAGE_SIZE,
        class_mode="binary",
        shuffle=False
    )

    val_dataset = val_datagen.flow_from_dataframe(
        dataframe=val_dataframe,
        directory=None,
        x_col="filepaths",
        y_col="labels",
        target_size=IMAGE_SIZE,
        class_mode="binary",
        shuffle=False
    )

    model_path = f"model_fold_{fold}.keras"
    best_model = ModelCheckpoint(filepath=model_path,
                                 monitor="val_accuracy",
                                 verbose=1,
                                 save_best_only=True,
                                 mode="max")
    model = create_model()

    trained_model = model.fit(train_dataset,
              epochs=308,
              validation_data=val_dataset,
              callbacks=[best_model],
              verbose=1)
    max_val_accuracy: float = max(trained_model.history['val_accuracy'])
    max_val_index: int = trained_model.history['val_accuracy'].index(max_val_accuracy)
    accuracy: tuple[float, float] = (max_val_accuracy, trained_model.history['accuracy'][max_val_index])
    models_accuracies.append(accuracy)
    fold += 1
best_accuracy: float = max(models_accuracies)
best_model: int = models_accuracies.index(best_accuracy) + 1
breast_cancer_classifier = load_model(f"model_fold_{best_model}.keras")

In [ ]:
import os
import numpy as np
from keras.utils import load_img, img_to_array
from IPython.display import Image

%store breast_cancer_classifier
%store best_accuracy


diretorio_path = 'imagensCancerMama/teste_dataset'
imagens = os.listdir(diretorio_path)
index = np.random.randint(0, len(imagens))
imagem = imagens[index]
imagem_path = os.path.join(diretorio_path, imagem)

print(best_accuracy)

test_image = load_img(imagem_path, target_size=(256, 256))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
resultado = breast_cancer_classifier.predict(test_image)
print(f"Resultado da predição: {resultado[0][0]}")
print(imagem)
print("É um câncer maligno" if resultado[0][0] >= 0.5 else "É um câncer benigno")
Image(filename=imagem_path)
